In [2]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - A warm-up challenge (~1h)

## Select features

🎯 Let's figure out the impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using our `orders` training_set, we will run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [3]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Select in a list which features to be used:

⚠️ Make sure no data leakage is created (i.e. selecting features that are derived from the target)

💡 To figure out the impact of `wait_time` and `delay_vs_expected` we need to control for the impact of other features, include in the list all features that may be relevant

In [5]:
x_var = ['wait_time','expected_wait_time','delay_vs_expected','number_of_products','number_of_sellers','price','freight_value','distance_seller_customer']
y_val = ['dim_is_five_star','dim_is_five_star']
orders.head(3)

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,0.0,delivered,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,0.0,delivered,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,0.0,delivered,1,0,5,1,1,159.90,19.22,514.130333


🕵🏻 Check the `multi-colinearity` of the features using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 

⚖️ Standardizing:

In [6]:
base_scaled = orders[x_var].copy()
for feature in base_scaled.columns:
    mu = base_scaled[feature].mean()
    sigma = base_scaled[feature].std()
    base_scaled[feature] = base_scaled[feature].apply(lambda x: (x-mu)/sigma)

In [7]:
base_scaled.head(3)

,wait_time,expected_wait_time,delay_vs_expected,number_of_products,number_of_sellers,price,freight_value,distance_seller_customer
0,-0.428002,-0.955662,-0.160462,-0.264595,-0.112544,-0.513802,-0.652038,-0.979475
1,0.100519,-0.499255,-0.160462,-0.264595,-0.112544,-0.086640,0.000467,0.429743
2,-0.322297,0.299456,-0.160462,-0.264595,-0.112544,0.111748,-0.164053,-0.145495


In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
# compute VIF factor for feature index 0
vif(base_scaled.values, 1)

1.5974166673318788

In [9]:
df = pd.DataFrame()

df["features"] = base_scaled.columns

df["vif_index"] = [vif(base_scaled.values, i) for i in range(base_scaled.shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)

,features,vif_index
0,wait_time,3.04
2,delay_vs_expected,2.43
6,freight_value,1.68
7,distance_seller_customer,1.60
1,expected_wait_time,1.60
3,number_of_products,1.37
5,price,1.21
4,number_of_sellers,1.10


## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [10]:
logit_one = smf.logit(formula='dim_is_one_star ~ wait_time + delay_vs_expected', data=orders).fit()
logit_one.params

Optimization terminated successfully.
         Current function value: 0.283270
         Iterations 7


Intercept           -3.169822
wait_time            0.059533
delay_vs_expected    0.071306
dtype: float64

In [11]:
prob = ['probability']
exp = ['odds']
for param in logit_one.params:
    exp.append(np.exp(param))
    prob.append(np.exp(param)/(1+np.exp(param)))
print(exp)
print(prob)

['odds', 0.042011058359046075, 1.0613404864394242, 1.0739097228874708]
['probability', 0.040317286483700934, 0.5148787856355983, 0.5178189344675446]


`Logit 5️⃣`

In [12]:
logit_five = smf.logit(formula='dim_is_five_star ~ wait_time + delay_vs_expected', data=orders).fit()
logit_five.params

Optimization terminated successfully.
         Current function value: 0.642767
         Iterations 7


Intercept            0.978416
wait_time           -0.046713
delay_vs_expected   -0.105488
dtype: float64

In [13]:
prob2 = ['probability']
exp2 = ['odds']
for param in logit_five.params:
    exp2.append(np.exp(param))
    prob2.append(np.exp(param)/(1+np.exp(param)))
print(exp2)
print(prob2)

['odds', 2.6602389984172508, 0.9543610532687616, 0.8998851539512686]
['probability', 0.7267937966803761, 0.48832381901621885, 0.47365239529333697]


💡 It's time to analyse the results of these two logistic regressions:

- Check their statistical significances with `p-values`
- Identify if there is any differences between `logit_one` and `logit_five` in terms of coefficient importances?

Through the answer below, we can see that delay_vs_expected influences five_star ratings even more than one_star ratings

> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...


## Logistic vs. Linear ?

👉 Compare:
- the regression coefficients obtained from the `Logistic Regression `
- with the regression coefficients obtained through a `Linear Regression` 
- on `review_score`, using the same features. 

⚠️ Check that both sets of coefficients  tell  "the same story".

In [14]:
features = ["review_score", "wait_time","delay_vs_expected", "price", "distance_seller_customer", 
       "number_of_products", "number_of_sellers"]

orders_scaled_features = orders.copy()
orders_scaled_features
orders_scaled_features = orders_scaled_features[features]

for feature in orders_scaled_features.columns[1:]:
    mu = orders_scaled_features[feature].mean()
    sigma = orders_scaled_features[feature].std()
    orders_scaled_features[feature] = orders_scaled_features[feature].apply(lambda x: (x-mu)/sigma)
    
orders_scaled_features.head()

,review_score,wait_time,delay_vs_expected,price,distance_seller_customer,number_of_products,number_of_sellers
0,4,-0.428002,-0.160462,-0.513802,-0.979475,-0.264595,-0.112544
1,4,0.100519,-0.160462,-0.086640,0.429743,-0.264595,-0.112544
2,5,-0.322297,-0.160462,0.111748,-0.145495,-0.264595,-0.112544
3,5,0.100519,-0.160462,-0.441525,2.054621,-0.264595,-0.112544
4,5,-1.062226,-0.160462,-0.562388,-0.959115,-0.264595,-0.112544


In [15]:
ols_model = smf.ols(formula = '''review_score ~ wait_time + delay_vs_expected + price + 
distance_seller_customer + number_of_products + number_of_sellers''', data = orders_scaled_features)

ols_model = ols_model.fit()

ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           review_score   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     2705.
Date:                Fri, 16 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:22:13   Log-Likelihood:            -1.5259e+05
No. Observations:               95872   AIC:                         3.052e+05
Df Residuals:                   95865   BIC:                         3.053e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    4.1555      0.004   1082.625      0.000       4.148       4.163
wait_time                   -0.4410      0.006    -71.316      0.000      -0.453      -0.429
delay_vs_expected           -0.0495      0.006     -8.706      0.000      -0.061      -0.038
price                       -0.0016      0.004     -0.400      0.689      -0.009       0.006
distance_seller_customer     0.0982      0.004     22.209      0.000       0.090       0.107
number_of_products          -0.1280      0.004    -31.573      0.000      -0.136      -0.120
number_of_sellers           -0.1314      0.004    -32.749      0.000      -0.139      -0.124
==============================================================================
Omnibus:                    18736.377   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            37607.679
Skew:                          -1.177   Prob(JB):                         0.00
Kurtosis:                       4.968   Cond. No.                         2.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""